# 챌린지. Image Classifier의 HyperParameter를 튜닝하기

In [ ]:
!pip install bayesian-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])

dataset  = torchvision.datasets.FashionMNIST("FashionMNIST/", download = True, train = True, transform = transform)
testset  = torchvision.datasets.FashionMNIST("FashionMNIST/", download = True, train = False, transform = transform)
labels_map = {0 : 'T-Shirt', 1 : 'Trouser', 2 : 'Pullover', 3 : 'Dress', 4 : 'Coat', 5 : 'Sandal', 6 : 'Shirt', 7 : 'Sneaker', 8 : 'Bag', 9 : 'Ankle Boot'}

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting FashionMNIST/FashionMNIST/raw/train-images-idx3-ubyte.gz to FashionMNIST/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting FashionMNIST/FashionMNIST/raw/train-labels-idx1-ubyte.gz to FashionMNIST/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting FashionMNIST/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to FashionMNIST/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting FashionMNIST/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to FashionMNIST/FashionMNIST/raw



In [ ]:
trainset, valset =  random_split(dataset, [50000, 10000])

In [ ]:
class FashionCNN(nn.Module):
    def __init__(self, dropout = 0.5):
        super(FashionCNN, self).__init__()

        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 10, kernel_size = 5),
            nn.MaxPool2d(2),
            nn.ReLU(),
            nn.Conv2d(10, 20, kernel_size = 5),
            nn.Dropout(dropout),
            nn.MaxPool2d(2),
            nn.ReLU(),
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(320, 50),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(50, 10),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(-1, 320)
        x = self.fc_layers(x)
        return x

In [ ]:
from bayes_opt import BayesianOptimization

def evaluate(epochs,  learning_rate, batch_size, dropout):
    epochs = int(epochs)
    batch_size = int(batch_size)
    trainloader = DataLoader(dataset = trainset, batch_size = batch_size, shuffle = True, num_workers = 2)
    valloader = DataLoader(dataset = valset, batch_size = batch_size, shuffle = True, num_workers = 2)
    testloader = DataLoader(dataset = testset, batch_size = batch_size, shuffle = True, num_workers = 2)
    
    model = FashionCNN(dropout).to(device)
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 
    criterion = nn.CrossEntropyLoss()
    
    for epoch in range(1, epochs+1):
        loss_value = 0
        for step, (x, y) in enumerate(trainloader):
            model.train()
            x, y = x.to(device), y.to(device)
            x = Variable(x.view(-1, 1, 28, 28))
            # y = Variable(y.view(-1))
            
            pred = model(x)
            loss = criterion(pred, y)
            loss_value += loss.item()
            optimizer.zero_grad()

            loss.backward()
            optimizer.step()
        
        correct = 0
        total = 0
        
        for step, (x, y) in enumerate(valloader):
            
            model.eval()
            with torch.no_grad():
                x = x.to(device)
                outputs = model(x)
                pred = torch.max(outputs, 1)[1].to(device)
                correct += (pred.detach().cpu() == y).sum()
                total += len(y)
            accuracy = correct * 100 / total
    return accuracy


cnn_bo = BayesianOptimization(evaluate, {'epochs':(5,10),
                                           'batch_size':(100,500),
                                           'learning_rate':(0.01, 0.1),
                                           'dropout':(0.1,0.7)})

cnn_bo.maximize(init_points=3, n_iter=5, acq='ei')

|   iter    |  target   | batch_... |  dropout  |  epochs   | learni... |
-------------------------------------------------------------------------


<ipython-input-6-91844eda8443>:52: DeprecationWarning: 
Passing acquisition function parameters or gaussian process parameters to maximize
is no longer supported, and will cause an error in future releases. Instead,
please use the "set_gp_params" method to set the gp params, and pass an instance
 of bayes_opt.util.UtilityFunction using the acquisition_function argument

  cnn_bo.maximize(init_points=3, n_iter=5, acq='ei')


| 1         | 78.77     | 490.0     | 0.5237    | 8.742     | 0.01308   |
| 2         | 66.22     | 299.3     | 0.5361    | 8.66      | 0.0288    |
| 3         | 9.86      | 140.2     | 0.699     | 5.932     | 0.06906   |
| 4         | 70.73     | 489.0     | 0.4256    | 7.785     | 0.03208   |
| 5         | 10.11     | 498.0     | 0.2476    | 9.984     | 0.08888   |
| 6         | 53.66     | 305.1     | 0.2849    | 8.924     | 0.01961   |
| 7         | 9.71      | 295.0     | 0.6943    | 5.0       | 0.07371   |
| 8         | 72.27     | 492.2     | 0.4885    | 5.993     | 0.02267   |


In [ ]:
epochs = 8
batch_size = 490
dropout = 0.5237
learning_rate = 0.01308

trainloader = DataLoader(dataset = trainset, batch_size = batch_size, shuffle = True, num_workers = 2)
valloader = DataLoader(dataset = valset, batch_size = batch_size, shuffle = True, num_workers = 2)
testloader = DataLoader(dataset = testset, batch_size = batch_size, shuffle = True, num_workers = 2)

model = FashionCNN(dropout).to(device)
model.train()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 
criterion = nn.CrossEntropyLoss()
loss_list = []
for epoch in range(1, epochs+1):
    loss_value = 0
    for step, (x, y) in enumerate(trainloader):
        model.train()
        x, y = x.to(device), y.to(device)
        x = Variable(x.view(-1, 1, 28, 28))
        pred = model(x)
        loss = criterion(pred, y)
        loss_value += loss.item()
        optimizer.zero_grad()

        loss.backward()
        optimizer.step()
    loss_list.append(loss_value)
    correct = 0
    total = 0

    for step, (x, y) in enumerate(valloader):

        model.eval()
        with torch.no_grad():
            x = x.to(device)
            outputs = model(x)
            pred = torch.max(outputs, 1)[1].to(device)
            correct += (pred.detach().cpu() == y).sum()
            total += len(y)
        accuracy = correct * 100 / total
    print("Epoch : ", epoch, ', Loss : ', loss.item(), 'Accuracy', accuracy.item() , '%')

Epoch :  1 , Loss :  1.6448500156402588 Accuracy 71.44000244140625 %
Epoch :  2 , Loss :  1.672964096069336 Accuracy 77.87999725341797 %
Epoch :  3 , Loss :  1.7977638244628906 Accuracy 79.61000061035156 %
Epoch :  4 , Loss :  1.6264352798461914 Accuracy 79.41000366210938 %
Epoch :  5 , Loss :  1.8159345388412476 Accuracy 79.8499984741211 %
Epoch :  6 , Loss :  1.560457468032837 Accuracy 79.52999877929688 %
Epoch :  7 , Loss :  1.8103816509246826 Accuracy 79.5 %
Epoch :  8 , Loss :  1.6094303131103516 Accuracy 80.61000061035156 %


In [ ]:
total = 0
correct = 0
for images, labels in testloader:
    images, labels = images.to(device), labels.to(device)            
    test = Variable(images.view(-1, 1, 28, 28))            
    outputs = model(test)            
    predictions = torch.max(outputs, 1)[1].to(device)
    correct += (predictions == labels).sum()            
    total += len(labels)

accuracy = correct * 100 / total
print('Test Accuracy : ', accuracy.item())

Test Accuracy :  80.02999877929688
